In [ ]:
import glob
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix
import os
import pandas as pd


In [3]:
import os
from pyhere import here
import sys
sys.path.append(str(here('01_level1')))
from level1_utils import run_level1

/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [2]:
subnum = '01'
mnum = 'model3'

base_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/'
data_path = os.path.join(base_path, 'bids_nifti_wface/')
behavior_path = os.path.join(base_path, 'behavioral_data/all_trials.csv')
out_path = os.path.join(data_path, 'derivatives/nilearn/glm/level1/%s_reg-rt%s'%(mnum, "0"))

In [4]:
run_level1(subnum, mnum, data_path, behavior_path, out_path, regress_rt=0, save_contrast = True)

A 'modulation' column was found in the given events data and is used.
***********************************************
Saving design matrix for sub-01 run-1
***********************************************
A 'modulation' column was found in the given events data and is used.
***********************************************
Saving design matrix for sub-01 run-2
***********************************************
A 'modulation' column was found in the given events data and is used.
***********************************************
Saving design matrix for sub-01 run-3
***********************************************
A 'modulation' column was found in the given events data and is used.
***********************************************
Saving design matrix for sub-01 run-4
***********************************************


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/_utils/glm.py:310: UserWarning: Matrix is singular at working precision, regularizing...
  warn('Matrix is singular at working precision, regularizing...')


A 'modulation' column was found in the given events data and is used.
***********************************************
Saving design matrix for sub-01 run-5
***********************************************
***********************************************
Running fixed effects GLM for all runs of sub-01
***********************************************
***********************************************
Saving GLM for sub-01
***********************************************
***********************************************
Running contrasts for sub-01
***********************************************


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/glm/first_level/first_level.py:652: UserWarning: One contrast given, assuming it for all 5 runs
  warn('One contrast given, assuming it for all %d runs' % n_runs)


***********************************************
Done saving contrasts for sub-01
***********************************************


In [ ]:
from level1_utils import make_level1_design_matrix, get_events

In [ ]:
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1', data_path, behavior_path, regress_rt = 0)
# plot_design_matrix(design_matrix)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1a', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix = make_level1_design_matrix(subnum, runnum, 'model2', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model3', data_path, behavior_path, regress_rt = 0, hrf_model=None)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
plot_design_matrix(design_matrix)